# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from maddpg_agent import MADDPGAgent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
multiagents = MADDPGAgent(agent_size=num_agents, state_size=state_size, action_size=action_size, random_seed=2)

In [6]:
env_info = env.reset(train_mode=True)[brain_name]

In [7]:
def maddpg(n_episodes=1000, max_t=1000, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        
        
        #state = env.reset()
        env_info = env.reset(train_mode=True)[brain_name]
        all_states = env_info.vector_observations                  # get the current state (for each agent)
        
        
        
        multiagents.reset()
        score = np.zeros(num_agents)
        for t in range(max_t):
            all_actions = np.concatenate(multiagents.act(obs_all_agents=all_states, noise=True),axis=0)
            #print(all_actions)
            env_info = env.step(all_actions)[brain_name]
            all_next_states = env_info.vector_observations
            all_rewards = env_info.rewards
            all_dones = env_info.local_done
            
            
            
            multiagents.step(all_states, all_actions, all_rewards, all_next_states, all_dones)
            all_states = all_next_states
            score += all_rewards
        scores_deque.append(np.max(score))
        scores.append(np.max(score))
        print('\rEpisode {}\tAverage Score: {:.2f}\tAgent1: {}\tAgent2: {}'.format(i_episode, np.mean(scores_deque), score[0], score[1]), end="")
        torch.save(multiagents.maddpg_agent[0].actor_local.state_dict(), 'checkpoint_agent1_actor.pth')
        torch.save(multiagents.maddpg_agent[0].critic_local.state_dict(), 'checkpoint_agent1_critic.pth')
        torch.save(multiagents.maddpg_agent[1].actor_local.state_dict(), 'checkpoint_agent2_actor.pth')
        torch.save(multiagents.maddpg_agent[1].critic_local.state_dict(), 'checkpoint_agent2_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores

scores = maddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

tensor([[[-6.3648, -1.5589,  3.3007, -0.9810,  6.8317,  5.8901,  3.3007,
          -0.9810, -5.8265, -1.7158,  5.3827, -1.9620,  6.8317,  5.6939,
           5.3827, -1.9620, -5.1769, -1.8524,  6.4965,  0.0000,  6.8317,
           5.3996,  6.4965,  0.0000],
         [-6.9295, -1.5589,  0.6682, -0.9810, -6.8317,  5.8901,  0.6682,
          -0.9810, -6.6679, -1.7158,  2.6159, -1.9620, -6.8317,  5.6939,
           2.6159, -1.9620, -6.2607, -1.8524,  4.0723,  0.0000, -6.8317,
           5.3996,  4.0723,  0.0000]],

        [[-5.8265, -1.7158,  5.3827, -1.9620,  6.8317,  5.6939,  5.3827,
          -1.9620, -5.1769, -1.8524,  6.4965,  0.0000,  6.8317,  5.3996,
           6.4965,  0.0000, -4.4900, -1.8522,  6.8685,  0.0000,  6.8317,
           5.0072,  6.8685,  0.0000],
         [-6.6679, -1.7158,  2.6159, -1.9620, -6.8317,  5.6939,  2.6159,
          -1.9620, -6.2607, -1.8524,  4.0723,  0.0000, -6.8317,  5.3996,
           4.0723,  0.0000, -5.6896, -1.8522,  5.7111,  0.0000, -6.8317,
        

TypeError: index_select(): argument 'index' must be Tensor, not int

In [ ]:
env.close()

In [ ]:
import numpy as np
test = np.random.rand(2,2)

In [ ]:
print(test)

In [ ]:
print(test[0])